In [67]:
import xgboost as xgb 
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_target = df['SUBCLASS']

data = df.drop(columns=['ID', 'SUBCLASS'])
df_test.drop(columns=['ID'], inplace=True)
df

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [2]:
disease_list = df.SUBCLASS.unique()
disease_list

array(['KIPAN', 'SARC', 'SKCM', 'KIRC', 'GBMLGG', 'STES', 'BRCA', 'THCA',
       'LIHC', 'HNSC', 'PAAD', 'OV', 'PRAD', 'UCEC', 'LAML', 'COAD',
       'ACC', 'LGG', 'LUSC', 'LUAD', 'CESC', 'PCPG', 'THYM', 'BLCA',
       'TGCT', 'DLBC'], dtype=object)

In [29]:
def seek1():
    count_df = df[df.SUBCLASS=='ACC'].apply(lambda x : x!='WT').agg(sum).reset_index(name='count')
    gene_list = count_df[count_df['count']>0]['index'].values
    print(len(gene_list))

In [ ]:
df[df.SUBCLASS=='SKCM'].apply(lambda x : x!='WT').columns


In [31]:
def get_gene_list2():
    genes = []
    gene_set = set()
    for disease in df["SUBCLASS"].unique():
        disease_df = df[df["SUBCLASS"] == disease]

        non_wt_counts = (disease_df != 'WT').sum()  # "WT"가 아닌 행 개수
        non_wt_counts_sorted = non_wt_counts.sort_values(ascending=False) #빈도 수가 높은 순서대로 정렬

        n = 20 # 추출한 상위 유전자 개수
        genes.extend(list(non_wt_counts_sorted[2:2+n].index))

        #print(f'{disease} - 전체 행의 개수 : {len(disease_df)}')
        dd = non_wt_counts_sorted[2:n].reset_index()
        gene_list1 = dd['index'].values
        gene_set.update(gene_list1)
        #print(gene_list1)
    return gene_set

In [30]:
def get_gene_list1():
    selected_genes = set()
    for d in disease_list:
        count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
        count_df = count_df[2:].reset_index(name='count')        
        gene_list = count_df[count_df['count']>10]['index'].values
        print(d, len(gene_list))
        selected_genes.update(gene_list)
    print(len(selected_genes))
    return selected_genes
glist = get_gene_list1()

KIPAN 49
SARC 4
SKCM 1091
KIRC 10
GBMLGG 33
STES 862
BRCA 154
THCA 3
LIHC 15
HNSC 81
PAAD 3
OV 3
PRAD 3
UCEC 681
LAML 5
COAD 244
ACC 19
LGG 6
LUSC 174
LUAD 139
CESC 31
PCPG 1
THYM 0
BLCA 38
TGCT 3
DLBC 2
1523


In [59]:
df

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [75]:
aa =  list(glist)
aa.append('SUBCLASS')
vis_df = df.loc[:, aa]
vis_df

,GOLGA4,MYH10,P2RY2,ESYT1,RNASEL,DCLRE1B,CHPF2,CENPE,KDM7A,PIKFYVE,...,NCAN,CERCAM,ADCY6,ATP6V0A4,PTCH1,HK2,EIF2AK3,B3GAT1,ABCA2,SUBCLASS
0,A699T,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,KIPAN
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,SARC
2,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,G1443V,WT,WT,WT,WT,SKCM
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,KIRC
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,GBMLGG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,R75R,WT,LUAD
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,LGG
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,COAD
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,TGCT


In [76]:
for c in vis_df.columns :
    if c != 'SUBCLASS':
        vis_df[c]  = vis_df[c].apply(lambda x : 0 if x=='WT' else 1 )
vis_df

,GOLGA4,MYH10,P2RY2,ESYT1,RNASEL,DCLRE1B,CHPF2,CENPE,KDM7A,PIKFYVE,...,NCAN,CERCAM,ADCY6,ATP6V0A4,PTCH1,HK2,EIF2AK3,B3GAT1,ABCA2,SUBCLASS
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,KIPAN
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,SARC
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,SKCM
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,KIRC
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,GBMLGG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,LUAD
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,LGG
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,COAD
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,TGCT


In [89]:
import matplotlib.pyplot as plt
import seaborn as sns
vis_df.set_index('SUBCLASS', inplace=True)
vis_df

,GOLGA4,MYH10,P2RY2,ESYT1,RNASEL,DCLRE1B,CHPF2,CENPE,KDM7A,PIKFYVE,...,NRXN2,NCAN,CERCAM,ADCY6,ATP6V0A4,PTCH1,HK2,EIF2AK3,B3GAT1,ABCA2
GOLGA4,1.000000,0.180403,0.041641,0.176559,0.115372,0.175214,0.098423,0.192205,0.081522,0.174211,...,0.156448,0.161432,0.086963,0.121328,0.172980,0.170585,0.161342,0.124279,0.066801,0.084876
MYH10,0.180403,1.000000,0.078651,0.176840,0.132364,0.122790,0.134330,0.185390,0.138046,0.149918,...,0.188379,0.125865,0.103340,0.148994,0.162398,0.153680,0.089277,0.102434,0.076574,0.134661
P2RY2,0.041641,0.078651,1.000000,0.074389,0.103873,0.099793,0.046888,0.088411,0.151368,0.098281,...,0.113070,0.122468,0.066170,0.091991,0.083647,0.135204,0.098627,0.102307,0.070806,0.078115
ESYT1,0.176559,0.176840,0.074389,1.000000,0.123784,0.134314,0.066911,0.097112,0.108089,0.133249,...,0.118128,0.117930,0.098938,0.087382,0.170770,0.107552,0.130546,0.054801,0.039557,0.114718
RNASEL,0.115372,0.132364,0.103873,0.123784,1.000000,0.164016,0.154020,0.166699,0.103820,0.145970,...,0.141186,0.142934,0.110132,0.152188,0.139007,0.167968,0.125434,0.091141,0.069537,0.109945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PTCH1,0.170585,0.153680,0.135204,0.107552,0.167968,0.121373,0.090332,0.176610,0.085485,0.167368,...,0.146410,0.124219,0.080671,0.127900,0.142587,1.000000,0.150432,0.119483,0.075569,0.066221
HK2,0.161342,0.089277,0.098627,0.130546,0.125434,0.123057,0.146590,0.140408,0.084657,0.103252,...,0.133694,0.125911,0.032091,0.131629,0.156307,0.150432,1.000000,0.073763,0.065716,0.073805
EIF2AK3,0.124279,0.102434,0.102307,0.054801,0.091141,0.110316,0.122176,0.155012,0.059538,0.143663,...,0.101991,0.140707,0.093529,0.122933,0.111898,0.119483,0.073763,1.000000,0.052629,0.077093
B3GAT1,0.066801,0.076574,0.070806,0.039557,0.069537,0.097572,0.063855,0.063690,0.113095,0.117983,...,0.121691,0.107464,0.046180,0.039971,0.035456,0.075569,0.065716,0.052629,1.000000,0.101578


In [93]:
vis_df.groupby('SUBCLASS').agg(sum)

,GOLGA4,MYH10,P2RY2,ESYT1,RNASEL,DCLRE1B,CHPF2,CENPE,KDM7A,PIKFYVE,...,NRXN2,NCAN,CERCAM,ADCY6,ATP6V0A4,PTCH1,HK2,EIF2AK3,B3GAT1,ABCA2
SUBCLASS,,,,,,,,,,,,,,,,,,,,,
ACC,3,0,2,1,1,0,0,3,2,2,...,5,2,0,2,0,2,3,1,1,2
BLCA,3,7,1,4,2,1,0,7,2,6,...,4,5,3,3,1,7,2,4,1,2
BRCA,9,15,1,7,4,1,1,13,2,11,...,15,7,3,3,6,9,5,4,1,5
CESC,3,7,0,2,1,1,0,5,0,4,...,4,4,3,2,2,2,5,2,1,7
COAD,10,11,3,5,3,2,3,11,1,16,...,5,7,2,2,7,17,4,3,1,0
DLBC,2,5,0,0,0,0,1,1,0,2,...,1,1,0,1,1,3,2,1,0,1
GBMLGG,4,2,3,1,0,2,1,4,5,3,...,1,2,2,2,3,0,3,2,1,1
HNSC,2,12,3,3,4,1,3,6,5,8,...,4,7,1,2,3,9,3,2,0,7
KIPAN,3,2,3,2,3,1,3,10,2,3,...,7,2,7,1,3,8,1,0,2,4


In [ ]:
selected_genes = set()
for d in disease_list:
    print(d)
    count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum).reset_index(name='count')
    gene_list = count_df[count_df['count']>10]['index'].values
    #print(len(gene_list))
    df = df[df.SUBCLASS==d].loc[:, list(glist)]
for c in df.columns[2:]:
    df[c] = df[c].apply(lambda x : 0 if x=='WT' else 1).agg(sum)
    selected_genes = selected_genes.union(gene_list)   
    da = df.iloc[0][2:]#.reset_index(name='count')
    da.plot()

In [39]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for c in total.columns:
    total[c] = encoder.fit_transform(total[c])



In [40]:
df = total[:6201]
df_test = total[6201:]



In [ ]:
df

In [ ]:
df_test

In [ ]:
from sklearn.preprocessing import LabelEncoder
X = df
y = df_target
encoder2 = LabelEncoder()
y = encoder2.fit_transform(y)
y


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156)
print(X_train.shape, X_test.shape)

In [45]:
dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)


#Multi-class classification

In [ ]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    'num_class': 26
}
num_boost_round = 1000

dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)
model = xgb.train(params=params, early_stopping_rounds=5, dtrain=dtrain, num_boost_round=num_boost_round,
                  evals=[(dtrain, 'train'), (dtest, 'valid')],
                  verbose_eval=2)

In [ ]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    'learning_rate': 0.05
}
num_boost_round = 1000

clf = xgb.XGBClassifier(n_estimators=num_boost_round,early_stopping_rounds=5, **params)
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)],
        verbose=2);

In [ ]:
y_true = y_test
y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)
y_true.shape, y_pred.shape, y_score.shape

In [ ]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

In [ ]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

In [ ]:
print(metrics.classification_report(y_true, y_pred))

In [ ]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

In [ ]:
clf.predict(df_test)

In [56]:
res = clf.predict(df_test)



In [57]:
res = encoder2.inverse_transform(res)

In [ ]:

df_test1 = pd.read_csv('test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df


In [59]:
res_df.to_csv('res07-2578-2.csv',index = False)

In [44]:
pred_probs = xgb_model.predict(dtest)


In [ ]:
import numpy as np
print(np.round(pred_probs[:10], 3))


In [58]:
preds= [round(x) if x > 0 else 0 for x in pred_probs]

In [ ]:
preds

In [60]:
for p in preds:
    if p<0:
        print(p)


In [ ]:
encoder.inverse_transform(preds)

In [35]:
model = RandomForestClassifier(
            n_estimators=50, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            criterion='entropy', ## 불순도 측도
            max_depth=65, ## 개별 나무의 최대 깊이
            #max_features='sqrt', ## 매 분리시 랜덤으로 뽑을 변수 개수
            #max_samples=1.0, ## 붓스트랩 샘플 비율 => 1이면 학습데이터를 모두 샘플링한다.
            #bootstrap=True, ## 복원 추출,  False이면 비복원 추출
            oob_score=True, ## Out-of-bag 데이터를 이용한 성능 계산
            random_state=100
).fit(X,y)


In [ ]:
## 예측
print(model.predict(X)[:3]) 
print()


In [ ]:
X[:3]

In [ ]:
## 성능 평가
print(model.oob_score_) ## Out-of-bag 성능 평가 점수
print('정확도 : ', model.score(X,y)) ## 테스트 성능 평가 점수(Accuracy)
print()
## 변수 중요도
print(model.feature_importances_)




In [ ]:
## 성능 평가
print(model.oob_score_) ## Out-of-bag 성능 평가 점수
print('정확도 : ', model.score(X,y)) ## 테스트 성능 평가 점수(Accuracy)
print()
## 변수 중요도
print(model.feature_importances_)




In [ ]:
res = model.predict(df_test)
res

In [ ]:
res = model.predict(df_test)
res

In [ ]:
df_test1 = pd.read_csv('test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df



In [24]:
res_df.to_csv('res04.csv',index = False)

In [ ]:
df[df.SUBCLASS == 'BRCA'].sum().sort_values(ascending=False).str.replace('WT','')

In [ ]:
df[df['SUBCLASS']=='DLBC']

In [ ]:
import numpy as np 
dfs = df[df['SUBCLASS']=='DLBC']
dfs.columns

In [ ]:
for c in dfs.columns:
    dfs[c] = np.where(dfs[c] == 'WT', 0, 1)
dfs

    

In [ ]:
dfs = df[df['SUBCLASS']=='DLBC']
dfs.apply(lambda x: x(if x=='WT' else x=1))

In [ ]:
for  x in df.iloc[0]:
    if x != 'WT': print(x)


In [ ]:
len(df_test.columns)

In [ ]:
len(df_test)

In [ ]:
df.colu